In [88]:
%serialconnect


Connecting to Serial /dev/ttyUSB3 baud=115200 
Ready.


In [79]:
%sendtofile --source ds3231.py


Sent 41 lines (2050 bytes) to ds3231.py.


In [81]:
import machine
i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4))
import ds3231

In [87]:
import time
for i in range(10):
    time.sleep(0.3)
    print(ds3231.jsepochtoisodate(ds3231.rtctojsepoch(i2c, True)))

.2017-11-19T16:48:35.000
2017-11-19T16:48:36.000
2017-11-19T16:48:37.000
2017-11-19T16:48:38.000
2017-11-19T16:48:39.000
2017-11-19T16:48:40.000
2017-11-19T16:48:41.000
2017-11-19T16:48:42.000
2017-11-19T16:48:43.000
2017-11-19T16:48:44.000


In [66]:
x = ds3231.isodatetojsepoch("2017-11-19T16:05:45.413Z")
print(x, 1511107545413)

1511107545413 1511107545413


In [ ]:
ds3231.jsepochtortc(x)   # set the value

In [80]:
%rebootdevice

repl is in normal command mode
[\r\x03\x03] b'\r\nMicroPython v1.9.2-443-g236297f4 on 2017-11-14; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\nPYB: soft reboot\r\nOSError: [Errno 2] ENOENT\r\nMicroPython v1.9.2-443-g236297f4 on 2017-11-14; ESP32 module with ESP32\r\nType "help()" for more information.\r\n>>> \r\n>>> \r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>'

In [2]:
import os
os.remove("main.py")


In [8]:
import machine
p = machine.Pin(13, machine.Pin.OUT)
p.value(1)

In [21]:
w = machine.PWM(machine.Pin(13), freq=1024, duty=40)



In [22]:
w.freq(10)
print(w)

PWM(13, freq=10, duty=40)


In [24]:
w.duty(2)

In [25]:

i2c = machine.I2C(scl=machine.Pin(5), sda=machine.Pin(4))


In [29]:
print(i2c.scan())

[87, 104, 119]


In [28]:
print(0x68, hex(87))

104 0x57


In [73]:
#
# Realtime clock code, handles isodates, javascriptepoch timestamps (in milliseconds), 
# and interface to the DS3231 as well as the internal RTC
# 

epoch1970 = 946684800000   # esp8266 has 2000 epoch
def rhex(h):  
    return (h>>4)*10 + (h&0x0f)
def dhex(v):  
    return chr(((v//10)<<4) + (v%10))

def rtctojsepoch(bwaitsec=False):
    r = i2c.readfrom_mem(0x68, 0x00, 7)
    year, month, day = rhex(r[6])+2000, rhex(r[5]), rhex(r[4])
    hour, minute, second = rhex(r[2]), rhex(r[1]), rhex(r[0])
    micropythonepoch = time.mktime((year, month, day, hour, minute, second, -1, -1))
    if bwaitsec:
        micropythonepoch += 1000
        while (r[0] == i2c.readfrom_mem(0x68, 0x00, 1)[0]):
            pass
    return micropythonepoch*1000 + epoch1970   # esp8266 has 2000 epoch


def rtctojsepoch():
    r = i2c.readfrom_mem(0x68, 0x00, 7)
    year, month, day = rhex(r[6])+2000, rhex(r[5]), rhex(r[4])
    hour, minute, second = rhex(r[2]), rhex(r[1]), rhex(r[0])
    micropythonepoch = time.mktime((year, month, day, hour, minute, second, -1, -1))
    return micropythonepoch*1000 + epoch1970   # esp8266 has 2000 epoch

def jsepochtortc(jsepoch):
    micropythonepoch = (jsepoch - epoch1970)//1000
    microsecond = jsepoch % 1000
    year, month, day, hour, minute, second, d1, d2 = time.localtime(micropythonepoch)
    i2c.writeto(0x68, chr(0) + dhex(second) + dhex(minute) + dhex(hour))
    i2c.writeto(0x68, chr(4) + dhex(day) + dhex(month) + dhex(year-2000))
    
def isodatetojsepoch(isodate):  # 2017-11-19T16:05:45.413Z
    #mtime = ure.match("(\d\d\d\d)-(\d\d)-(\d\d)[T ](\d\d)[:c](\d\d)[:c](\d\d)[\.d](\d\d\d)", isodate)
    #year, month, day, hour, minute, second, microsecond = int(mtime.group(1)), int(mtime.group(2)), int(mtime.group(3)), int(mtime.group(4)), int(mtime.group(5)), int(mtime.group(6)), int(mtime.group(7)) 
    year, month, day = int(isodate[:4]), int(isodate[5:7]), int(isodate[8:10])
    hour, minute, second, microsecond = int(isodate[11:13]), int(isodate[14:16]), int(isodate[17:19]), int(isodate[20:23])
    micropythonepoch = time.mktime((year, month, day, hour, minute, second, -1, -1))
    return micropythonepoch*1000 + microsecond + epoch1970

def jsepochtoisodate(jsepoch):
    micropythonepoch = (jsepoch - epoch1970)//1000
    microsecond = jsepoch % 1000
    year, month, day, hour, minute, second, d1, d2 = time.localtime(micropythonepoch)
    return "{0:04d}-{1:02d}-{2:02d}T{3:02d}:{4:02d}:{5:02d}.{6:03d}".format(year, month, day, hour, minute, second, microsecond)
    


In [74]:
def rtctojsepoch(bwaitsec=False):
    r = i2c.readfrom_mem(0x68, 0x00, 7)
    year, month, day = rhex(r[6])+2000, rhex(r[5]), rhex(r[4])
    hour, minute, second = rhex(r[2]), rhex(r[1]), rhex(r[0])
    micropythonepoch = time.mktime((year, month, day, hour, minute, second, -1, -1))
    if bwaitsec:
        micropythonepoch += 1000
        while (r[0] == i2c.readfrom_mem(0x68, 0x00, 1)[0]):
            pass
    return micropythonepoch*1000 + epoch1970   # esp8266 has 2000 epoch


In [78]:
import time
for i in range(10):
    time.sleep(0.8)
    print(rtctojsepoch(True))
    

1511109993000
.1511109994000
1511109995000
1511109996000
1511109997000
1511109998000
1511109999000
1511110000000
1511110001000
1511110002000


In [33]:
print(rtctojsepoch())

946685444000


In [55]:
x = isodatetojsepoch("2017-11-19T16:05:45.413Z")
print(x, 1511107545413)


1511107545413


In [35]:
x = isodatetojsepoch("2017-11-19T16:05:45.413Z")
print(x)


1511107545413


In [36]:
print(jsepochtoisodate(x))


2017-11-19T16:05:45.413


In [38]:
jsepochtortc(x)

In [49]:
print(jsepochtoisodate(rtctojsepoch()))

2017-11-19T16:07:07.000


In [89]:
import machine
machine.Pin(13)
help(machine)

object <module 'umachine'> is of type module
  __name__ -- umachine
  mem8 -- <8-bit memory>
  mem16 -- <16-bit memory>
  mem32 -- <32-bit memory>
  freq -- <function>
  reset -- <function>
  unique_id -- <function>
  idle -- <function>
  disable_irq -- <function>
  enable_irq -- <function>
  time_pulse_us -- <function>
  Timer -- <class 'Timer'>
  WDT -- <class 'WDT'>
  Pin -- <class 'Pin'>
  Signal -- <class 'Signal'>
  TouchPad -- <class 'TouchPad'>
  ADC -- <class 'ADC'>
  DAC -- <class 'DAC'>
  I2C -- <class 'I2C'>
  PWM -- <class 'PWM'>
  SPI -- <class 'SoftSPI'>
  UART -- <class 'UART'>


In [90]:
x = machine.TouchPad(machine.Pin(32))


In [93]:
import time
for i in range(30):
    print(x.read())
    time.sleep(0.5)
    

1709
1712
1712
1712
1710
1710
1713
1713
1712
1682
1702
1704
1707
1708
1706
1706
1708
1705
1704
1701
1702
1697
1704
1703
1709
1711
1706
1705
1710
1715
